In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=менеджер&no_magic=true&L_save_area=true&items_on_page=20'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page

Cтраница №32, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=31&hhtmFrom=vacancy_search_list
Cтраница №33, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=32&hhtmFrom=vacancy_search_list
Cтраница №34, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=33&hhtmFrom=vacancy_search_list
Cтраница №35, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

Cтраница №63, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=62&hhtmFrom=vacancy_search_list
Cтраница №64, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=63&hhtmFrom=vacancy_search_list
Cтраница №65, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=64&hhtmFrom=vacancy_search_list
Cтраница №66, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

Cтраница №94, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=93&hhtmFrom=vacancy_search_list
Cтраница №95, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=94&hhtmFrom=vacancy_search_list
Cтраница №96, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=95&hhtmFrom=vacancy_search_list
Cтраница №97, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

In [6]:
len(result)

2000

In [7]:
result

[{'vacancy_name': 'Чат-менеджер',
  'vacancy_salary': '25\u202f000 – 65\u202f000 руб.',
  'min_salary': 25000,
  'max_salary': 65000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://adsrv.hh.ru/click?b=369066&c=30&meta=P63GoVnpEm_RObgKe-xm3Y_yQg-5vTXbGlYjAJfuMmoOntyeJTsli2v6jzYmmDP33vKyMSKQCgnCQiCFScTK1TaUSWV0Ft9xAlJKn07eZqqlnMg14f_16_hw9yGgBQAiylz0KjHcpqwXOKZ3rz9k8C63WzD94Gj9v15LGItz7_QZldWUfzJOoGry0eHRWX2m14MigXvelfUDvOsMc_xkzQ%3D%3D&place=35&clickType=link_to_vacancy',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Менеджер по продажам (Малоэтажное строительство)',
  'vacancy_salary': 'от 100\u202f000 руб.',
  'min_salary': 100000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77555675?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Старший менеджер по обслуживанию',
  'vacancy_salary': 'от 42\u202f500 руб.',
  'min_salary': 42500,
  'max_sal